In [106]:
%%pyspark
#df = spark.read.load.('abfss://bronze@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer.txt', format='text', header='true')
df = spark.read.option("header",True).csv('abfss://bronze@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer.txt')

display(df.limit(10))

StatementMeta(testhmacspark, 25, 52, Finished, Available)

SynapseWidget(Synapse.DataFrame, 66c05e26-dfcc-4d45-8734-3b348fbacc1d)

In [107]:
import hashlib
def fun_lst_hash(size, secret):

    lst = list(range(0,size))
    lst_hash =[]
    for item in lst:

        value = str(item).rjust(len(str(size))-1, '0') + secret
        encoded_value = str.encode(value)
        hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()

        lst_hash.append(str(hash_obj_sha256))
        lst_hash.sort()
    return lst_hash

secret10 = TokenLibrary.getSecret('test-hmacmask-akv', 'secret10', 'AzureKeyVaultHMAC')
lst_10_hash = fun_lst_hash(10, secret10)
#
secret100 = TokenLibrary.getSecret('test-hmacmask-akv', 'secret100', 'AzureKeyVaultHMAC')
lst_100_hash = fun_lst_hash(100, secret100)
#
secret1000 = TokenLibrary.getSecret('test-hmacmask-akv', 'secret1000', 'AzureKeyVaultHMAC')
lst_1000_hash = fun_lst_hash(1000, secret1000)
#
secret10000 = TokenLibrary.getSecret('test-hmacmask-akv', 'secret10000', 'AzureKeyVaultHMAC')
lst_10000_hash = fun_lst_hash(10000, secret10000)
#
secret100000 = TokenLibrary.getSecret('test-hmacmask-akv', 'secret100000', 'AzureKeyVaultHMAC')
lst_100000_hash = fun_lst_hash(100000, secret100000)
#
secrets_all = [secret10, secret100, secret1000, secret10000, secret100000]
lst_hash_all = [lst_10_hash, lst_100_hash, lst_1000_hash, lst_10000_hash, lst_100000_hash]


StatementMeta(testhmacspark, 25, 53, Finished, Available)

In [112]:
def mask_func(col_val):
    col_val_pfe_raw = ""
    dummy_val_string_array = ""
    for i, val in enumerate(col_val):
        if val.isnumeric():
            dummy_val_string_array += str(val)

        if len(dummy_val_string_array) == 5 or i == len(col_val) - 1:
            # either max length of substring is reached or end of string is reached, start obfuscating string
            value = dummy_val_string_array + secrets_all[len(dummy_val_string_array)-1]
            encoded_value = str.encode(value)
            hash_obj_sha256 = hashlib.sha256(encoded_value).hexdigest()
            index_pfe = lst_hash_all[len(dummy_val_string_array)-1].index(hash_obj_sha256)
            value_pfe = str(index_pfe).rjust(len(dummy_val_string_array), '0')
            col_val_pfe_raw += value_pfe
            dummy_val_string_array = ""

    col_val_pfe = ""
    non_numeric = 0
    for j, val in enumerate(col_val):
        if not(val.isnumeric()):
            col_val_pfe += col_val[j]
            non_numeric += 1
        else:
            col_val_pfe += col_val_pfe_raw[j-non_numeric]
    
    return col_val_pfe


phone = "123-456-7890"

phone_pfe = mask_func(phone)
print(phone_pfe)
#print(lst_100000_hash)

StatementMeta(testhmacspark, 25, 58, Finished, Available)

420-085-5464


In [113]:
# https://tunetotech.com/Post.aspx?post=9
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
#
mask_func_udf = udf(mask_func, StringType())
df2=df.withColumn('Phone', mask_func_udf(df.Phone))

display(df2.limit(10))

StatementMeta(testhmacspark, 25, 59, Finished, Available)

SynapseWidget(Synapse.DataFrame, fd70514e-8638-4799-81ed-ba5e0f9530ea)

In [53]:
df2.write.mode("overwrite").option("header","true").csv('abfss://gold@testhmacmaskstor.dfs.core.windows.net/SalesLT.Customer_masked.txt')

StatementMeta(testhmacspark, 24, 53, Finished, Available)